__ORDER BY__

last clause in the query and it sorts the results returned by the rest of the query

__Dates__

-> DATE - yyyy-[m]m-[d]d. 

-> DATETIME - like the DATE format but with added time in the end

__EXTRACT__

if we want to extract the part of the date like year of day we can do this by EXTRACT

In [1]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'smiling-audio-287409-77f38c77b232.json'

In [2]:
from google.cloud import bigquery


In [3]:
# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "nhtsa_traffic_fatalities" dataset
dataset_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [4]:
tables = list(client.list_tables(dataset))

In [5]:
for table in tables:
    print(table.table_id)

accident_2015
accident_2016
cevent_2015
cevent_2016
damage_2015
damage_2016
distract_2015
distract_2016
drimpair_2015
drimpair_2016
factor_2015
factor_2016
maneuver_2015
maneuver_2016
nmcrash_2015
nmcrash_2016
nmimpair_2015
nmimpair_2016
nmprior_2015
nmprior_2016
parkwork_2015
parkwork_2016
pbtype_2015
pbtype_2016
person_2015
person_2016
safetyeq_2015
safetyeq_2016
vehicle_2015
vehicle_2016
vevent_2015
vevent_2016
vindecode_2015
vindecode_2016
violatn_2015
violatn_2016
vision_2015
vision_2016
vsoe_2015
vsoe_2016


In [6]:
table_ref = dataset_ref.table("accident_2015")
table = client.get_table(table_ref)

In [7]:
client.list_rows(table, max_results = 10).to_dataframe().sample(n=10)

C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  """Entry point for launching an IPython kernel.
C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:1: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  """Entry point for launching an IPython kernel.


,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
7,19,Iowa,190069,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,1,2015-04-26 19:00:00+00:00
1,19,Iowa,190233,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,1,2015-11-01 00:30:00+00:00
4,19,Iowa,190231,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,0,2015-10-31 04:49:00+00:00
2,19,Iowa,190179,1,1,0,0,0,2,2,...,1,0,None,0,None,0,None,1,0,2015-05-04 16:18:00+00:00
9,19,Iowa,190046,1,1,0,0,0,2,2,...,99,0,None,0,None,0,None,1,1,2015-03-29 02:34:00+00:00
0,19,Iowa,190204,1,1,0,0,0,1,1,...,2,0,None,0,None,0,None,1,1,2015-09-11 20:20:00+00:00
5,19,Iowa,190151,1,1,0,0,0,2,2,...,99,0,None,0,None,0,None,1,0,2015-08-13 09:09:00+00:00
8,19,Iowa,190257,1,1,0,0,0,3,3,...,99,0,None,0,None,0,None,1,0,2015-12-04 12:42:00+00:00
6,19,Iowa,190054,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,1,2015-03-16 00:00:00+00:00
3,19,Iowa,190248,1,1,0,0,0,4,4,...,99,0,None,0,None,0,None,2,0,2015-11-17 12:26:00+00:00


Q1 - Let's use the table to determine how the number of accidents varies with the day of the week. 

Since:

the ___consecutive_number___ column contains a unique ID for each accident, and

the ___timestamp_of_crash___ column contains the date of the accident in DATETIME format,


we can:

EXTRACT the day of the week (as ___day_of_week___ in the query below) from the ___timestamp_of_crash___ column, 

GROUP BY the day of the week, before we __COUNT__ the consecutive_number column to determine the number of accidents for each day of the week.

Then we sort the table with an ORDER BY clause, so the days with the most accidents are returned first.

In [8]:
query_accidents =  """
            SELECT COUNT(1) AS num_accidents, 
                   EXTRACT (DAYOFWEEK FROM timestamp_of_crash) AS day_of_week
                   
            FROM `bigquery-public-data.nhtsa_traffic_fatalities.accident_2015`
            
            GROUP BY day_of_week
            ORDER BY num_accidents DESC
"""

In [9]:
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**9)
query_job = client.query(query_accidents, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
accidents_by_day = query_job.to_dataframe()

# Print the DataFrame
accidents_by_day


,num_accidents,day_of_week
0,5659,7
1,5298,1
2,4916,6
3,4460,5
4,4182,4
5,4038,2
6,3985,3


The World Bank has made tons of interesting education data available through BigQuery. Run the following cell to see the first few rows of the `international_education` table from the `world_bank_intl_education` dataset.

In [11]:
# Construct a reference to the "world_bank_intl_education" dataset
dataset_ref = client.dataset("world_bank_intl_education", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "international_education" table
table_ref = dataset_ref.table("international_education")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the "international_education" table
client.list_rows(table, max_results=10).to_dataframe().sample(n=10, replace = False) 


########## USING .sample(n=10) we can get random values  ############

C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Cannot use bqstorage_client if max_results is set, reverting to fetching data with the tabledata.list endpoint.
  
C:\Users\user\anaconda4\envs\tensorflow gpu\lib\site-packages\ipykernel_launcher.py:14: PyarrowMissingWarning: Converting to a dataframe without pyarrow installed is often slower and will become unsupported in the future. Please install the pyarrow package.
  


,country_name,country_code,indicator_name,indicator_code,value,year
9,Uzbekistan,UZB,"Population, ages 15-64 (% of total)",SP.POP.1564.TO.ZS,6.769146e+01,2016
5,"Congo, Dem. Rep.",COD,Population of the official age for secondary e...,SP.SEC.TOTL.IN,1.084283e+07,2016
8,Uzbekistan,UZB,"Enrolment in pre-primary education, female (nu...",SE.PRE.ENRL.FE,3.046210e+05,2016
6,"Congo, Dem. Rep.",COD,UIS: Percentage of population age 25+ with at ...,UIS.EA.1T6.AG25T99.F,4.994458e+01,2016
2,China,CHN,Population of the official age for secondary e...,SP.SEC.TOTL.FE.IN,4.199033e+07,2016
7,Turkmenistan,TKM,Theoretical duration of primary education (years),SE.PRM.DURS,4.000000e+00,2016
0,Central African Republic,CAF,GDP at market prices (current US$),NY.GDP.MKTP.CD,1.756125e+09,2016
3,China,CHN,Population of the official entrance age to sec...,UIS.SAP.23.GPV.G1.F,6.993104e+06,2016
4,Comoros,COM,Population growth (annual %),SP.POP.GROW,2.311191e+00,2016
1,China,CHN,Population of the official age for lower secon...,SP.SEC.LTOT.MA.IN,2.406085e+07,2016



The value in the `indicator_code` column describes what type of data is shown in a given row.  

One interesting indicator code is `SE.XPD.TOTL.GD.ZS`, which corresponds to "Government expenditure on education as % of GDP (%)".

### 1) Government expenditure on education

Which countries spend the largest fraction of GDP on education?  

To answer this question, consider only the rows in the dataset corresponding to indicator code `SE.XPD.TOTL.GD.ZS`, and write a query that returns the average value in the `value` column for each country in the dataset between the years 2010-2017 (including 2010 and 2017 in the average). 

Requirements:
- Your results should have the country name rather than the country code. You will have one row for each country.
- The aggregate function for average is **AVG()**.  Use the name `avg_ed_spending_pct` for the column created by this aggregation.
- Order the results so the countries that spend the largest fraction of GDP on education show up first.


In [17]:
query_exp = """
        SELECT country_name, AVG(value) AS avg_ed_spending_pct
        FROM `bigquery-public-data.world_bank_intl_education.international_education`
        WHERE indicator_code = "SE.XPD.TOTL.GD.ZS" and year >= 2010 and year <= 2017
        GROUP BY country_name
        ORDER BY avg_ed_spending_pct DESC
"""

In [19]:
# Set up the query (cancel the query if it would use too much of 
# your quota, with the limit set to 1 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
country_spend_pct_query_job = client.query(query_exp, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
country_spending_results = country_spend_pct_query_job.to_dataframe()

# View top few rows of results
print(country_spending_results.head())

            country_name  avg_ed_spending_pct
0                   Cuba            12.837270
1  Micronesia, Fed. Sts.            12.467750
2        Solomon Islands            10.001080
3                Moldova             8.372153
4                Namibia             8.349610


### 2) Identify interesting codes to explore

The last question started by telling you to focus on rows with the code `SE.XPD.TOTL.GD.ZS`. But how would you find more interesting indicator codes to explore?

There are 1000s of codes in the dataset, so it would be time consuming to review them all. But many codes are available for only a few countries. When browsing the options for different codes, you might restrict yourself to codes that are reported by many countries.

Write a query below that selects the indicator code and indicator name for all codes with at least 175 rows in the year 2016.

Requirements:
- You should have one row for each indicator code.
- The columns in your results should be called `indicator_code`, `indicator_name`, and `num_rows`.
- Only select codes with 175 or more rows in the raw database (exactly 175 rows would be included).
- To get both the `indicator_code` and `indicator_name` in your resulting DataFrame, you need to include both in your **SELECT** statement (in addition to a **COUNT()** aggregation). This requires you to include both in your **GROUP BY** clause.
- Order from results most frequent to least frequent.

In [20]:
code_count_query = """
                SELECT indicator_code,indicator_name, COUNT(1) AS num_rows
                FROM `bigquery-public-data.world_bank_intl_education.international_education`
                WHERE year = 2016
                GROUP BY indicator_code,indicator_name
                HAVING COUNT(1)>= 175
                ORDER BY COUNT(1) DESC

"""

# Set up the query
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
code_count_query_job = client.query(code_count_query, job_config=safe_config)

# API request - run the query, and return a pandas DataFrame
code_count_results = code_count_query_job.to_dataframe()

# View top few rows of results
print(code_count_results.head())

      indicator_code                      indicator_name  num_rows
0        SP.POP.GROW        Population growth (annual %)       232
1        SP.POP.TOTL                   Population, total       232
2     IT.NET.USER.P2     Internet users (per 100 people)       223
3  SP.POP.0014.MA.IN         Population, ages 0-14, male       213
4  SP.POP.0014.TO.ZS  Population, ages 0-14 (% of total)       213
